### Applied DYNOTEARS Algorithm

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

feature_matrix = pd.read_parquet(
    '../data/feature_matrix.parquet',
)

In [9]:
feature_matrix.reset_index(drop = True)

,USOIL,TBILL,COPPER,DXY,HYS,BEI,VIX,Cluster_1,Cluster_2,Cluster_3
0,0.819004,0.052340,0.133165,1.152554,1.989326,1.55,3.030134,-0.448489,-0.322804,-0.990628
1,0.800399,0.099827,0.145027,1.157060,2.092157,1.56,2.962175,-0.438204,-0.265458,-0.950945
2,0.745985,0.096079,0.138594,1.153274,1.994548,1.53,3.024806,-0.521034,-0.306381,-1.016384
3,0.737325,0.082241,0.110705,1.143227,2.088216,1.50,3.218476,-0.651195,-0.383645,-1.144683
4,0.736815,0.079552,0.103964,1.148113,2.200708,1.48,3.296207,-0.663432,-0.412488,-1.158419
...,...,...,...,...,...,...,...,...,...,...
2533,1.072875,0.811986,0.478793,1.130212,-0.147062,2.36,2.858193,0.588235,0.115614,0.557341
2534,1.014370,0.820790,0.462450,1.133574,-0.056174,2.35,2.794228,0.631540,0.142374,0.588528
2535,1.046593,0.833450,0.511300,1.130091,-0.164576,2.36,2.890927,0.605878,0.162064,0.468238
2536,1.052318,0.833167,0.477163,1.132822,-0.158989,2.35,2.924773,0.627010,0.208329,0.453092


In [21]:
from causalnex.structure.dynotears import from_pandas_dynamic
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

# 2. DYNOTEARS 모델 학습
# lambda_w, lambda_a: 정규화 계수 (L1 penalty). 너무 복잡한 그래프가 나오면 이 값을 키우세요.
sm = from_pandas_dynamic(
    feature_matrix.resample('W-FRI').last().reset_index(drop = True),
    p=4,
    lambda_w=0.0,
    lambda_a=0.0,
    w_threshold=0.1
)

In [22]:
# 1. 모든 에러 없이 에지(Edges) 가져오기
all_edges = sm.edges

# 2. Intra-slice (오늘 -> 오늘) 추출
intra_edges = [edge for edge in all_edges if "_lag0" in edge[0] and "_lag0" in edge[1]]

# 3. Inter-slice (어제 -> 오늘) 추출
inter_edges = [edge for edge in all_edges if "_lag1" in edge[0] and "_lag0" in edge[1]]

print("--- Intra-slice Edges (Today -> Today) ---")
for u, v in intra_edges:
    # 가중치(Weight)까지 보고 싶다면 sm.get_edge_data(u, v) 사용
    w = sm.get_edge_data(u, v)['weight']
    print(f"{u} ---> {v} | Weight: {w:.4f}")

print("\n--- Inter-slice Edges (Yesterday -> Today) ---")
for u, v in inter_edges:
    w = sm.get_edge_data(u, v)['weight']
    print(f"{u} ---> {v} | Weight: {w:.4f}")

--- Intra-slice Edges (Today -> Today) ---
USOIL_lag0 ---> TBILL_lag0 | Weight: 0.1505
USOIL_lag0 ---> HYS_lag0 | Weight: -0.7774
USOIL_lag0 ---> BEI_lag0 | Weight: 0.5291
USOIL_lag0 ---> VIX_lag0 | Weight: -0.2584
USOIL_lag0 ---> Cluster_1_lag0 | Weight: 0.1943
TBILL_lag0 ---> HYS_lag0 | Weight: -1.0017
TBILL_lag0 ---> BEI_lag0 | Weight: 0.2580
TBILL_lag0 ---> Cluster_1_lag0 | Weight: 0.1980
COPPER_lag0 ---> USOIL_lag0 | Weight: 0.4114
COPPER_lag0 ---> HYS_lag0 | Weight: 0.1288
COPPER_lag0 ---> BEI_lag0 | Weight: 0.5191
COPPER_lag0 ---> VIX_lag0 | Weight: -0.3065
COPPER_lag0 ---> Cluster_1_lag0 | Weight: 0.6061
COPPER_lag0 ---> Cluster_2_lag0 | Weight: 0.5622
COPPER_lag0 ---> Cluster_3_lag0 | Weight: 0.1390
DXY_lag0 ---> USOIL_lag0 | Weight: 0.2441
DXY_lag0 ---> TBILL_lag0 | Weight: 0.8208
DXY_lag0 ---> COPPER_lag0 | Weight: -0.8759
DXY_lag0 ---> HYS_lag0 | Weight: 0.6928
DXY_lag0 ---> BEI_lag0 | Weight: 0.2590
DXY_lag0 ---> VIX_lag0 | Weight: -2.0575
DXY_lag0 ---> Cluster_1_lag0 | We

In [23]:
inter_df = pd.DataFrame([
    {'From (Yesterday)': u.replace('_lag1', ''),
     'To (Today)': v.replace('_lag0', ''),
     'Weight': sm.get_edge_data(u, v)['weight']}
    for u, v in inter_edges
])

print(inter_df.sort_values(by='Weight', ascending=False).head(10))

   From (Yesterday) To (Today)    Weight
6             TBILL      TBILL  0.920051
26              BEI        BEI  0.800348
1             USOIL        HYS  0.785921
0             USOIL      USOIL  0.686344
29              VIX        VIX  0.660478
19              DXY        DXY  0.658522
33        Cluster_2        HYS  0.648979
20              DXY        HYS  0.639482
38        Cluster_3  Cluster_3  0.584231
11           COPPER     COPPER  0.578182
